# FairWorkflows execution demo

## Define the steps of your workflow
Each step should be its own function. Mark the function as such with the @fairstep decorator.

In [1]:
%cd ..

from fairworkflows import is_fairworkflow, is_fairstep, FairStep, FairWorkflow

/home/dsmits/projects/FAIR-workbench/fairworkflows


In [2]:
@is_fairstep(label='Addition')
def add(a:float, b:float) -> float:
    """Adding up numbers!"""
    return a + b

In [3]:
@is_fairstep(label='Subtraction', is_manual_task=True)
def verify(a: float) -> bool:
    """Confirm that you like this result"""
    pass

## Define your workflow using @fairworkflow
Now write a function which describes your workflow. Mark this function with the @fairworkflow decorator.

In [4]:
@is_fairworkflow(label='My Workflow')
def my_workflow(in1, in2):
    """
    Add two numbers together and confirm the result
    """
    t1 = add(in1, in2)
    t2 = verify(t1)
    return t2

## Create an instance of your workflow and display it

In [6]:
fw = FairWorkflow.from_function(my_workflow)
type(fw)

fairworkflows.fairworkflow.FairWorkflow

## Execute your workflow using .execute()
Set num_threads greater than 1 if you wish to exploit parallelisation in your workflow. The retrospective provenance is also returned as a (nano) Publication object, that can optionally be published.

In [7]:
result, prov = fw.execute(1, 4)
result

INFO:root:Starting Manual Step Assistant
INFO:root:Please go to http://localhost:8000 to perform the manual step
127.0.0.1 - - [09/Mar/2021 11:20:26] "GET / HTTP/1.1" 200 -
INFO:root:Manual step has been completed.


{'out1': True}

### Retrospective prov
A WorkflowRetroProv object is returned along with the result of the execution.

In [13]:
type(prov)

fairworkflows.prov.WorkflowRetroProv

In [14]:
print(prov)

Workflow retrospective provenance.
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix prov: <http://www.w3.org/ns/prov#> .

[] a pplan:Bundle,
        prov:Collection ;
    prov:wasDerivedFrom <http://purl.org/np/RAPHCa5UMvnF40Jt7wimjUdxDMrMepUUqfAB9BTiA1tmY#plan> .




### Retrospective prov for each step
You can iterate through a WorkflowRetroProv object to get the StepRetroProv objects for each step. Print these to see the RDF they contain (input/output variable values, start and end datetime of the step's execution etc.) 

In [17]:
for sp in prov:
    print(sp)

Step retrospective provenance.
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a pplan:Activity ;
    pplan:correspondsToStep <http://purl.org/np/RAyTHTDs-Qqqu4EVYQjva4JoxWxpCUApESi1AGh93oQkU#step> ;
    prov:endedAtTime "2021-03-05T13:14:00.084623"^^xsd:dateTime ;
    prov:startedAtTime "2021-03-05T13:14:00.084616"^^xsd:dateTime ;
    prov:used [ a pplan:Entity ;
            rdfs:label "a" ;
            pplan:correspondsToVariable <http://purl.org/np/RAyTHTDs-Qqqu4EVYQjva4JoxWxpCUApESi1AGh93oQkU#a> ;
            rdf:value 1 ],
        [ a pplan:Entity ;
            rdfs:label "b" ;
            pplan:correspondsToVariable <http://purl.org/np/RAyTHTDs-Qqqu4EVYQjva4JoxWxpCUApESi1AGh93oQkU#b> ;
            rdf:value 4 ],
        [ a pplan:Entity ;
            rdfs:label "out1"

### Publish the retrospective provenance
You can use the .publish_as_nanopub() method as with FairStep and FairWorkflow objects. This publishes a nanopub per step and one for the whole workflow, mirroring the prospective RDF.

In [16]:
prov.publish_as_nanopub(use_test_server=True)

Published to http://purl.org/np/RAS5YybkJDH1go2SOQBqfoz1po5nHiyzRmNVgOW_CjiWs
Published concept to http://purl.org/np/RAS5YybkJDH1go2SOQBqfoz1po5nHiyzRmNVgOW_CjiWs#fairstepprov
Published to http://purl.org/np/RA7UtS-evZuhRiXnBVfCHElgzbnqAn3p7Sfyuuwh7bvP8
Published concept to http://purl.org/np/RA7UtS-evZuhRiXnBVfCHElgzbnqAn3p7Sfyuuwh7bvP8#fairstepprov
Published to http://purl.org/np/RAwoC5X6IiDrlsKS_ilgNN_DVGrr2THL30DeElETlC_ac
Published concept to http://purl.org/np/RAwoC5X6IiDrlsKS_ilgNN_DVGrr2THL30DeElETlC_ac#fairstepprov
Published to http://purl.org/np/RACERKeebWRMrV3-i0rgYGWtE3-7AGIVfHmWs8Gl34Zro
Published concept to http://purl.org/np/RACERKeebWRMrV3-i0rgYGWtE3-7AGIVfHmWs8Gl34Zro#fairstepprov
Published to http://purl.org/np/RAoV5hX_Hz9Ce_hJtmYNxGAqzrfM45KzokI_Pdr8HOAgQ
Published concept to http://purl.org/np/RAoV5hX_Hz9Ce_hJtmYNxGAqzrfM45KzokI_Pdr8HOAgQ#fairworkflowprov


{'nanopub_uri': 'http://purl.org/np/RAoV5hX_Hz9Ce_hJtmYNxGAqzrfM45KzokI_Pdr8HOAgQ',
 'concept_uri': 'http://purl.org/np/RAoV5hX_Hz9Ce_hJtmYNxGAqzrfM45KzokI_Pdr8HOAgQ#fairworkflowprov'}

The last nanopub (whose URI ends in #fairworkflowprov) contains the links to all of the individual step retrospective provenances.

## Provide semantic annotations for input and output variables
If you wish to specify semantic types for the inputs/outputs to a step, you can do so in the arguments to the decorator.
For example, if you have an input parameter 'a', you can write a='http://www.example.org/distance' to assign that (semantic) type to a. As output of functions is not named in python, you can specify the same but with 'out1', 'out2' etc. See the following example:

In [16]:
@is_fairstep(label='Addition', a='http://www.example.org/distance', returns='http://www.example.org/mass')
def add(a:float, b:float) -> float:
    return a + b

If we now look at the RDF generated for the step, we will see that input parameter 'a' and the step output ('out1') both have the (additional) semantic types specified.

In [17]:
# ACTIONS:
# Add language and version to nanopubs (i.e. what the description is written in)
print(add._fairstep)

Step URI = http://www.example.org/unpublished-add
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .

_:N55c6db2a96b4414499dd088141690acb {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition" ;
        dc:description """@is_fairstep(label='Addition', a='http://www.example.org/distance', returns='http://www.example.org/mass')
def add(a:float, b:float) -> float:
    return a + b
""" ;
        dc:language [ a schema:ComputerLanguage ;
                rdfs:label "python" ;
                rdfs:seeAlso <https://www.wikidata.org/wiki/Q28865> ;
                owl:versionInfo "3.7.9.final.0" ] ;
        pplan:hasInputVar [ a pplan:Variable,
                    <http://www.example.org/distance> ;
                rdfs:label "a

### Specify more than one semantic type for a parameter
You can provide a list of URIs if you want to specify several semantic types for e.g. parameter 'a':

In [18]:
@is_fairstep(label='Addition', a=['http://www.example.org/distance', 'http://www.example.org/number'])
def another_step(a:float, b:float) -> float:
    """Add two numbers together"""
    return a + b

In [19]:
print(another_step._fairstep)

Step URI = http://www.example.org/unpublished-another_step
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .

_:Nab534dffa6f1415cbbd25881f8d8e11d {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition" ;
        dc:description """@is_fairstep(label='Addition', a=['http://www.example.org/distance', 'http://www.example.org/number'])
def another_step(a:float, b:float) -> float:
    \"\"\"Add two numbers together\"\"\"
    return a + b
""" ;
        dc:language [ a schema:ComputerLanguage ;
                rdfs:label "python" ;
                rdfs:seeAlso <https://www.wikidata.org/wiki/Q28865> ;
                owl:versionInfo "3.7.9.final.0" ] ;
        pplan:hasInputVar [ a pplan:Variable,
                    <http://ww

You can check the programming language that was used for writing the step:

In [19]:
print(another_step._fairstep.language)

LinguisticSystem with type=https://schema.org/ComputerLanguage, label=python, seeAlso=https://www.wikidata.org/wiki/Q28865, versionInfo=3.7.9.final.0


## Semantic types for function producing multiple outputs
Provide 'out' with a tuple of the same length as the number of function outputs. You can use None for any you do not wish to assign a particular semantic type to.

In [20]:
from typing import Tuple
@is_fairstep(label='Addition and subtraction', returns=('http://www.example.org/distance', 'http://www.example.org/number'))
def another_step(a:float, b:float) -> Tuple[float, float]:
    return a + b, a - b

In [21]:
print(another_step._fairstep)

Step URI = http://www.example.org/unpublished-another_step
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .

_:N1345fcd9251f4f8b86ee8e59bbab507d {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition and subtraction" ;
        dc:description """@is_fairstep(label='Addition and subtraction', returns=('http://www.example.org/distance', 'http://www.example.org/number'))
def another_step(a:float, b:float) -> Tuple[float, float]:
    return a + b, a - b
""" ;
        dc:language [ a schema:ComputerLanguage ;
                rdfs:label "python" ;
                rdfs:seeAlso <https://www.wikidata.org/wiki/Q28865> ;
                owl:versionInfo "3.7.9.final.0" ] ;
        pplan:hasInputVar [ a pplan:Variable ;
           

As before, you may provide a list of URIs for each output. If you do not want to provide semantic types for a particular output, simply pass None:

In [22]:
from typing import Tuple
@is_fairstep(label='Addition and subtraction', returns=(['http://www.example.org/distance', 'http://www.example.org/number'], None))
def another_step(a:float, b:float) -> Tuple[float, float]:
    """This step returns an addition and a subtraction of its inputs"""
    return a + b, a - b

In [23]:
print(another_step._fairstep)

Step URI = http://www.example.org/unpublished-another_step
@prefix bpmn: <http://dkm.fbk.eu/index.php/BPMN2_Ontology#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pplan: <http://purl.org/net/p-plan#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .

_:Nd65e996b1d9546a4899eaff4131e159f {
    [] a bpmn:ScriptTask,
            pplan:Step ;
        rdfs:label "Addition and subtraction" ;
        dc:description """@is_fairstep(label='Addition and subtraction', returns=(['http://www.example.org/distance', 'http://www.example.org/number'], None))
def another_step(a:float, b:float) -> Tuple[float, float]:
    \"\"\"This step returns an addition and a subtraction of its inputs\"\"\"
    return a + b, a - b
""" ;
        dc:language [ a schema:ComputerLanguage ;
                rdfs:label "python" ;
                rdfs:seeAlso <https://www.wikidata.org/wiki/Q28865> ;
                owl:vers